## Example Bulk Si sp3s
### Symmetry adapted linear combinations
This example is essentially the same as the example of bulk silicon sp3 but include one additional orbitals, the higher 3s orbital, which is often included in the tight-binding parameterization of Si. 

Compare to the previous example ($3\times A_1$, $E (2-fold)$, $T_1 (3-fold)$, $4 \times T_2 (3-fold)$), the 4 additional $3s$ orbital form $2\times A_1$ and $T_2$. 

In [ ]:
from automaticTB.examples import get_Si_structure_2s2p3s
from automaticTB.functions import get_namedLCs_from_nncluster
import numpy as np

bulksi = get_Si_structure_2s2p3s()
si_environment = bulksi.nnclusters[0]
named_lcs = get_namedLCs_from_nncluster(si_environment)
for nlc in named_lcs:
    print(f"\"{nlc.name}\"")

### Free values of interaction
There we call the function that will give us the global independent parameter
```python
get_combined_equation_from_structure
```
from its result, we can obtain all the free independent interaction parameter in the structure. After these interaction are determined, we can supply values to these parameters. Here the values are taken from the paper of Klimeck et al. 2000 (NN VB) model. Notice that when suppling the value, we should notice the difference between the transition. In Klimeck's paper, the states on the right are positioned at (a/2, a/2, a/2) but here the state on the right are positioned at (a/2, -a/2, -a/2). It is also important to notice that the definition of the parameter in the paper is V = 4 Hij. For example: $V(s,s) = 4\langle saR | H | scR \rangle$. Therefore, it is necessary to divide by 4 for the interaction parameter.

Finally, we plot the band structure. it can be seem that the valence band agree in general very well with the Si band structure, while the conduction band deviate slight more. Dispersion along K-G is also problematic along the conduction band, so is the energy at K point for the valence band.

In [ ]:
from automaticTB.functions import (
    get_combined_equation_from_structure, 
    get_tbModel_from_structure_interactions_overlaps
)
from automaticTB.tightbinding import TightBindingModel
from automaticTB.tools import find_RCL
from automaticTB.properties import Kpath, Kline, BandStructureResult

combined_equation = get_combined_equation_from_structure(bulksi)

interaction_values = np.array([
    8.23164,  #  1 > Si     3s -> Si     3s @ (  0.00,  0.00,  0.00) 
   -3.3179,   #  2 > Si     2s -> Si     2s @ (  0.00,  0.00,  0.00) 
    1.67862,  #  3 > Si    2py -> Si    2py @ (  0.00,  0.00,  0.00) 
    0,        #  4 > Si     3s -> Si     2s @ (  1.36, -1.36, -1.36) 
    7.2505/4, #  5 > Si     3s -> Si    2px @ (  1.36, -1.36, -1.36) 
   -9.5990/4, #  6 > Si     2s -> Si     2s @ (  1.36, -1.36, -1.36) 
    0,        #  7 > Si     3s -> Si     3s @ (  1.36, -1.36, -1.36) 
    7.1423/4, #  8 > Si     2s -> Si    2px @ (  1.36, -1.36, -1.36) 
    0,        #  9 > Si     2s -> Si     3s @ (  1.36, -1.36, -1.36) 
   -7.1423/4, # 10 > Si    2px -> Si     2s @ (  1.36, -1.36, -1.36) 
   -4.7757/4, # 11 > Si    2px -> Si    2pz @ (  1.36, -1.36, -1.36)
    1.6955/4, # 12 > Si    2px -> Si    2px @ (  1.36, -1.36, -1.36) 
   -7.2505/4, # 13 > Si    2px -> Si     3s @ (  1.36, -1.36, -1.36)
])
# the divide by 4 is necessary because of the definition of V term, see P. Vogl et al. 1983 page 4, around equation 7)

interaction_pairs = combined_equation.solve_interactions_to_InteractionPairs(interaction_values)
model = get_tbModel_from_structure_interactions_overlaps(bulksi, interaction_pairs)


rcl = find_RCL(bulksi.cell)
path = Kpath(rcl, [
        Kline("L", np.array([0.5,0.5,0.5]), "G", np.array([0.0,0.0,0.0])),
        Kline("G", np.array([0.0,0.0,0.0]), "X", np.array([0.5,0.0,0.5])),
        Kline("X", np.array([0.5,0.0,0.5]), "U", np.array([5/8,1/4,5/8])),
        Kline("K", np.array([3/8,3/8,3/4]), "G", np.array([0.0,0.0,0.0]))
    ]
)
# reference for DFT calculated band structure: 
# http://lampx.tugraz.at/~hadley/ss1/semiconductors/silicon_bandstructure.php
bandresult = BandStructureResult.from_tightbinding_and_kpath(model, path)
tmp_filename = "Si_NN_VB.pdf"
bandresult.plot_data(tmp_filename)

def print_energy(tbmodel: TightBindingModel, kpos):
    kpos = np.array(kpos)
    e, _ = tbmodel.solveE_at_k(kpos)
    result = []
    for ee in e:
        result.append(f"{ee:>10.5f}")
    print(",".join(result))

print_energy(model, [0.5,0.5,0.5])
print_energy(model, [0.0,0.0,0.0])
print_energy(model, [0.5,0.0,0.5])
print_energy(model, [3/8,3/8,3/4])